## 2024년 2분기 ~ 4분기 읍면동별 임대료 시세 스크래핑

In [115]:
url = "https://golmok.seoul.go.kr/region/selectRentalPrice.json"

In [116]:
headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Mobile Safari/537.36",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
}

In [117]:
payload = {
  'stdrYyCd': 2024,
  'stdrSlctQu': 'beforeQu',
  'stdrQuCd': 4,
  'stdrMnCd': 202412,
  'selectTerm': 'quarter',
  'svcIndutyCdL': 'CS000000',
  'svcIndutyCdM': 'all',
  'stdrSigngu': 11,
  'selectInduty': 1,
  'infoCategory': 'rent'
}

In [118]:
import requests
import json
import pandas as pd

response = requests.post(url, headers=headers, data=payload)
response

<Response [200]>

In [119]:
rent_price = pd.DataFrame(response.json())
rent_price

,GBN_CD,NM,GUBUN,BF1_FST_FLOOR,BF2_FST_FLOOR,BF3_FST_FLOOR,BF1_EX_FLOOR,BF2_EX_FLOOR,BF3_EX_FLOOR,BF1_TOT_FLOOR,BF2_TOT_FLOOR,BF3_TOT_FLOOR
0,11,서울시 전체,si,150596,150268,153199,121547,120113,121129,136071,135190,137164
1,11110,종로구,gu,230674,223395,222029,160559,156781,155799,195617,190088,188914
2,11110515,청운효자동,dong,188588,201922,209717,115520,111876,106875,152054,156899,158296
3,11110530,사직동,dong,235516,219567,232820,132729,116110,134017,184123,167839,183418
4,11110540,삼청동,dong,153171,172621,187508,,,,153171,172621,187508
...,...,...,...,...,...,...,...,...,...,...,...,...
446,11740650,성내2동,dong,138906,141403,139056,110612,89392,86800,124759,115398,112928
447,11740660,성내3동,dong,131230,124472,127454,106118,94398,93217,118674,109435,110335
448,11740685,길동,dong,140377,139363,146998,80584,74186,81063,110481,106774,114030
449,11740690,둔촌1동,dong,,,,,,,,,


In [120]:
rent_price.rename(columns={
    'GBN_CD' : '행정_구역_코드',
    'NM' : '행정_구역_명',
    'GUBUN' : '행정_구역_분류',
    'BF1_FST_FLOOR' : '2024년_2분기_1층',
    'BF2_FST_FLOOR' : '2024년_3분기_1층',
    'BF3_FST_FLOOR' : '2024년_4분기_1층',
    'BF1_EX_FLOOR' : '2024년_2분기_1층_외',
    'BF2_EX_FLOOR' : '2024년_3분기_1층_외',
    'BF3_EX_FLOOR' : '2024년_4분기_1층_외',
    'BF1_TOT_FLOOR' : '2024년_2분기_총계',
    'BF2_TOT_FLOOR' : '2024년_3분기_총계',
    'BF3_TOT_FLOOR' : '2024년_4분기_총계',
}, inplace=True
)

In [121]:
rent_price.head()

,행정_구역_코드,행정_구역_명,행정_구역_분류,2024년_2분기_1층,2024년_3분기_1층,2024년_4분기_1층,2024년_2분기_1층_외,2024년_3분기_1층_외,2024년_4분기_1층_외,2024년_2분기_총계,2024년_3분기_총계,2024년_4분기_총계
0,11,서울시 전체,si,150596,150268,153199,121547,120113,121129,136071,135190,137164
1,11110,종로구,gu,230674,223395,222029,160559,156781,155799,195617,190088,188914
2,11110515,청운효자동,dong,188588,201922,209717,115520,111876,106875,152054,156899,158296
3,11110530,사직동,dong,235516,219567,232820,132729,116110,134017,184123,167839,183418
4,11110540,삼청동,dong,153171,172621,187508,,,,153171,172621,187508


## 함수로 만들기

In [122]:
def create_payload(stdrYyCd, stdrQuCd, stdrMnCd):
  payload = {
    'stdrYyCd': {stdrYyCd},
    'stdrSlctQu': 'beforeQu',
    'stdrQuCd': {stdrQuCd},
    'stdrMnCd': {stdrMnCd},
    'selectTerm': 'quarter',
    'svcIndutyCdL': 'CS000000',
    'svcIndutyCdM': 'all',
    'stdrSigngu': 11,
    'selectInduty': 1,
    'infoCategory': 'rent'
  }
  return payload

In [123]:
def rename_columns_for_period(df, year_quarter_):
    rename_dict = {
        'GBN_CD': '행정_구역_코드',
        'NM': '행정_구역_명',
        'GUBUN': '행정_구역_분류'
    }

    # 각 분기에 대한 컬럼명 추가
    for j, (year, quarter) in enumerate(year_quarter_, start=1):
        rename_dict[f'BF{j}_FST_FLOOR'] = f'{year}년_{quarter}분기_1층'
        rename_dict[f'BF{j}_EX_FLOOR'] = f'{year}년_{quarter}분기_1층_외'
        rename_dict[f'BF{j}_TOT_FLOOR'] = f'{year}년_{quarter}분기_총계'

    # 원본을 보호하기 위해 copy() 사용
    df_renamed = df.copy()
    df_renamed.rename(columns=rename_dict, inplace=True)
    return df_renamed

In [124]:
def scraping_rent_price(url, headers, payload):
  response = requests.post(url, headers=headers, data=payload)
  rent_price = pd.DataFrame(response.json())
  return rent_price

## 2019년 1분기 ~ 2024년 4분기 읍면동별 임대료 시세 전체 스크래핑

In [130]:
year_quarter_pairs = [
    (2024, 4, 202412),  # 2024년 4분기 ~ 2023년 2분기
    (2024, 1, 202412),  # 2024년 1분기 ~ 2023년 3분기
    (2023, 2, 202312),  # 2023년 2분기 ~ 2022년 4분기
    (2022, 3, 202212),  # 2022년 3분기 ~ 2022년 1분기
    (2021, 4, 202112),  # 2021년 4분기 ~ 2021년 2분기
    (2021, 1, 202112),  # 2021년 1분기 ~ 2020년 3분기
    (2020, 2, 202012),  # 2020년 2분기 ~ 2019년 4분기
    (2019, 3, 201912),  # 2019년 3분기 ~ 2019년 1분기
]

rent_prices = []

for year, quarter, month in year_quarter_pairs:
    payload = create_payload(year, quarter, month)  # payload 생성
    rent_price = scraping_rent_price(url, headers, payload)  # 스크래핑 요청
    rent_prices.append(rent_price)  # 결과 저장

In [131]:
rent_prices[0].head()

,GBN_CD,NM,GUBUN,BF1_FST_FLOOR,BF2_FST_FLOOR,BF3_FST_FLOOR,BF1_EX_FLOOR,BF2_EX_FLOOR,BF3_EX_FLOOR,BF1_TOT_FLOOR,BF2_TOT_FLOOR,BF3_TOT_FLOOR
0,11,서울시 전체,si,150596,150268,153199,121547,120113,121129,136071,135190,137164
1,11110,종로구,gu,230674,223395,222029,160559,156781,155799,195617,190088,188914
2,11110515,청운효자동,dong,188588,201922,209717,115520,111876,106875,152054,156899,158296
3,11110530,사직동,dong,235516,219567,232820,132729,116110,134017,184123,167839,183418
4,11110540,삼청동,dong,153171,172621,187508,,,,153171,172621,187508


In [132]:
year_quarter = [
    [(2024, 2),
    (2024, 3),
    (2024, 4)],
    [(2023, 3),
    (2023, 4),
    (2024, 1)],
    [(2022, 4),
    (2023, 1),
    (2023, 2)],
    [(2022, 1),
    (2022, 2),
    (2022, 3)],
    [(2021, 2),
    (2021, 3),
    (2021, 4)],
    [(2020, 3),
    (2020, 4),
    (2021, 1)],
    [(2019, 4),
    (2020, 1),
    (2020, 2)],
    [(2019, 1),
    (2019, 2),
    (2019, 3)],
]

In [133]:
rent_new_prices = []

for rent_price, year_quarter_ in zip(rent_prices, year_quarter):
    rent_new_prices.append(rename_columns_for_period(rent_price, year_quarter_))

In [134]:
rent_new_prices[5].head()

,행정_구역_코드,행정_구역_명,행정_구역_분류,2020년_3분기_1층,2020년_4분기_1층,2021년_1분기_1층,2020년_3분기_1층_외,2020년_4분기_1층_외,2021년_1분기_1층_외,2020년_3분기_총계,2020년_4분기_총계,2021년_1분기_총계
0,11,서울시 전체,si,140260,140213,140868,108221,104417,104182,124241,122315,122525
1,11110,종로구,gu,205895,199091,198296,137462,134928,136242,171678,167010,167269
2,11110515,청운효자동,dong,152656,150627,162329,99025,96136,93692,125841,123382,128011
3,11110530,사직동,dong,202244,195313,196087,105045,108733,109826,153645,152023,152957
4,11110540,삼청동,dong,187342,192051,190705,76618,76391,101202,131980,134221,145953


## 리스트에 있는 데이터프레임 하나로 합치기
* 행정_구역_코드, 행정_구역_명, 행정_구역_분류 키로 합치기

In [ ]:
from functools import reduce

# 공통 키 설정
merge_keys = ["행정_구역_코드", "행정_구역_명", "행정_구역_분류"]

# reduce()를 이용해 데이터프레임을 순차적으로 병합
# reduce는 여러 개의 값을 하나로 줄이는 함수
# 분기별 누락된 데이터도 남겨야하므로 outer 방식 사용
merged_df = reduce(lambda left, right: pd.merge(left, right, on=merge_keys, how="outer"), rent_new_prices)

merged_df

## 데이터프레임 CSV 파일로 변환

In [136]:
merged_df.head()

,행정_구역_코드,행정_구역_명,행정_구역_분류,2024년_2분기_1층,2024년_3분기_1층,2024년_4분기_1층,2024년_2분기_1층_외,2024년_3분기_1층_외,2024년_4분기_1층_외,2024년_2분기_총계,...,2020년_2분기_총계,2019년_1분기_1층,2019년_2분기_1층,2019년_3분기_1층,2019년_1분기_1층_외,2019년_2분기_1층_외,2019년_3분기_1층_외,2019년_1분기_총계,2019년_2분기_총계,2019년_3분기_총계
0,11,서울시 전체,si,150596,150268,153199,121547,120113,121129,136071,...,124141,132986,133888,133363,99721,100630,100707,116354,117259,117035
1,11110,종로구,gu,230674,223395,222029,160559,156781,155799,195617,...,172671,217272,228660,225458,152261,157835,158852,184767,193248,192155
2,11110515,청운효자동,dong,188588,201922,209717,115520,111876,106875,152054,...,131133,137481,144858,154347,73863,77012,91222,105672,110935,122785
3,11110530,사직동,dong,235516,219567,232820,132729,116110,134017,184123,...,153017,177492,172166,190743,124438,117668,111830,150965,144917,151287
4,11110540,삼청동,dong,153171,172621,187508,,,,153171,...,124398,197561,188928,161827,279017,288015,227825,238289,238472,194826


In [137]:
merged_df.to_csv("rent_price_data.csv", index=False, encoding="utf-8-sig")